In [1]:
# Import libraries
import glob

import pandas as pd
import numpy as np


In [2]:
# List all daily report files for ship position...
joined_list_pos = glob.glob("data/position_day*.csv") 

# ...and ship data
joined_list_ship = glob.glob("data/ship_data_day*.csv") 

# Create unified dataframes
df_positions = pd.concat(map(pd.read_csv, joined_list_pos), ignore_index=True)
df_ships_data = pd.concat(map(pd.read_csv, joined_list_ship), ignore_index=True)


In [3]:
# Define new column names
positions_columns = ['COG', 'Latitude', 'Longitude', 'NavigationalStatus', 'PositionAccuracy',
                    'RateOfTurn', 'SOG', 'TrueHeading', 'MMSI', 'ShipName', 'UTC_Time']

ships_columns = ['CallSign', 'Destination', 'ETA_Day', 'ETA_Hour', 'ETA_Minute', 'ETA_Month', 'IMO', 'ShipName',
                'ShipType', 'MMSI', 'Latitude', 'Longitude', 'UTC_Time']


In [4]:
# Rename columns

df_positions.drop("Unnamed: 0", axis = 1, inplace = True)
df_ships_data.drop("Unnamed: 0", axis = 1, inplace = True)

df_positions = df_positions.set_axis(positions_columns, axis = 1)
df_ships_data = df_ships_data.set_axis(ships_columns, axis = 1)


In [5]:
# Load MMSI Country Codes into a df...
country_codes_df = pd.read_csv("data/MMSI Country Codes.csv", delimiter=";", index_col=0, dtype=str)

# ...And convert to dictionary
country_codes = pd.Series(country_codes_df.Country.values, index=country_codes_df.index).to_dict()


In [6]:
# Get all information that can ID a ship from both AIS message types
positions_id = df_positions[["MMSI", "ShipName"]].drop_duplicates().astype(str)
ships_id = df_ships_data[["MMSI", "ShipName", "IMO", "ShipType"]].drop_duplicates().astype(str)

# Merge all that information
all_ships = positions_id.merge(ships_id, how="outer", on=["MMSI", "ShipName"])

# Get each ship's country flag, which can be obtained from the first 3 digits of each MMSI
all_ships["Flag"] = all_ships["MMSI"].str[:3].astype(int).map(country_codes)


In [7]:
# Filter out repeated ships

all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "374024000") & (all_ships["ShipType"] == "89")].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "314764000") & (all_ships["IMO"].isnull())].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "352002699") & (all_ships["ShipType"] == "89")].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "352524000") & (all_ships["IMO"].isnull())].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "636021195") & (all_ships["IMO"].isnull())].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "240484800") & (all_ships["ShipName"] == "nan")].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "240265600") & (all_ships["ShipName"] == "nan")].index)
all_ships = all_ships.drop(all_ships[(all_ships["MMSI"] == "240159808") & (all_ships["ShipName"] == "nan")].index)


In [8]:
# Fill in ship type nulls with value 99
all_ships["ShipType"].fillna(99, inplace=True)

# Change ship types to readable values
conditions = [
    (all_ships["ShipType"].astype(int) == 99),
    ((all_ships["ShipType"].astype(int) >= 80) & (all_ships["ShipType"].astype(int) < 90)),
    ((all_ships["ShipType"].astype(int) >= 70) & (all_ships["ShipType"].astype(int) < 80)),
    ((all_ships["ShipType"].astype(int) >= 60) & (all_ships["ShipType"].astype(int) < 70)),
    (all_ships["ShipType"].astype(int) == 52),
    ((all_ships["ShipType"].astype(int) >= 30) & (all_ships["ShipType"].astype(int) < 40)),
    (all_ships["ShipType"].astype(int) == 0)
]

values = ["Unknown", "Tanker", "Cargo Ship", "Passanger Ship", "Tug", "Generic Vessel", "Fishing"]

all_ships['ShipType'] = np.select(conditions, values)

# Export all ships for dashboard visalisations
all_ships.to_csv("./Data/all_ships.csv")

In [9]:
# Drop columns not to be used in final project
df_ships_data.drop(['CallSign', "Destination", "ETA_Day", "ETA_Hour", "ETA_Minute", "ETA_Month", "ShipType"], axis=1, inplace=True)
df_positions.drop(['COG', "NavigationalStatus", "PositionAccuracy", "RateOfTurn", "SOG", "TrueHeading"], axis=1, inplace=True)


In [10]:
# Select only oil tankers, cargo ships and unknown type ships
potential_tankers = all_ships.loc[all_ships["ShipType"].isin(["Cargo Ship", "Unknown", "Tanker"])]

# Select only known oil tankers to csv for dashboard visualisations
only_tankers = potential_tankers.loc[potential_tankers["ShipType"] == "Tanker"]
only_tankers.to_csv("./Data/only_tankers.csv")


In [11]:
# Homogenise MMSIs to string type
df_positions = df_positions.astype({"MMSI" : str})
df_ships_data = df_ships_data.astype({"MMSI" : str})

# Select only those ships that are tankers, cargo ships or unknown
tankers_positions = df_positions[df_positions['MMSI'].isin(potential_tankers["MMSI"].unique())]
tankers_ship_data = df_ships_data[df_ships_data['MMSI'].isin(potential_tankers["MMSI"].unique())]


In [12]:
# Merge to get all available information about all ships
only_flag_mmsi = potential_tankers[["MMSI", "Flag", "ShipType"]]

tankers_positions = pd.merge(tankers_positions, potential_tankers, on=['MMSI', 'ShipName'], how='left')
tankers_ship_data = pd.merge(tankers_ship_data, only_flag_mmsi, on=['MMSI'], how='left')

# Create a single dataframe with all messages and corresponding ship information
all_messages = pd.concat([tankers_positions, tankers_ship_data], axis=0)

# Save as CSV file for dashboard visualisations
all_messages.to_csv("./Data/All_Messages.csv", index=False)


In [13]:
# Convert message timestamp to datetime object
all_messages["start"] = pd.to_datetime(all_messages["UTC_Time"].str[:-4], format='%Y-%m-%d %H:%M:%S.%f %z')
all_messages.drop("UTC_Time", axis=1, inplace=True)


In [14]:
# Create a function to calculate the end of the Popup showing
def end_timestamp(df):
    output = []
    
    # Loop through all unique ship license plates
    for i in df["MMSI"].unique():
        subset = df.loc[df["MMSI"] == i]
        
        # Prepare the subset dataframe by sorting values by timestamp, creating the end column and reseting the 
        # index
        subset = subset.sort_values(by = "start", ascending=True)
        subset["end"] = 0
        subset.reset_index(inplace=True)
        
        # Iterate all rows in the subset
        for i in range(len(subset)):           
        
            # If the row is the last instance of the ship seen, delete the popup after 1 day
            if i == len(subset) - 1:
                subset.loc[i, "end"] = subset.loc[i, "start"] + pd.Timedelta(days = 1)
            # If the row is the not last instance of the ship seen, delete the popup when a new message is available
            else:
                subset.loc[i, "end"] = subset.loc[i + 1, "start"]
                
        output.append(subset)

    # Merge the results and return
    result = pd.concat(output) 
    result.drop("index", axis=1, inplace=True) 
    return result
     
# Apply function and add to general dataset
end_time_df = end_timestamp(all_messages[["MMSI", "start"]])
all_messages = pd.merge(all_messages, end_time_df, on=['MMSI', 'start'], how='left')


In [15]:
# Prepare the GeoJSON file with the fields required
all_messages["type"] = "Point"
all_messages["coordinates"] = [[lat, lon] for lat, lon in zip(all_messages["Longitude"], all_messages["Latitude"])]
all_messages.drop(["Latitude", "Longitude"], axis=1, inplace=True)

# Export JSON file for Leaflet map creation
all_messages.to_json("./Data/All_Messages.json", orient="records")
